# training

> Training loop

In [ ]:
#|default_exp initialization

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from AIsaac.utils import *
from AIsaac.dataloaders import *
from AIsaac.models import *

from datetime import datetime, timedelta
import torchvision.transforms.functional as TF,torch.nn.functional as F
import math, time

import matplotlib.pyplot as plt
import matplotlib as mpl
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd , numpy as np
from torcheval.metrics import MulticlassAccuracy,Mean
from torch.optim.lr_scheduler import ExponentialLR

import dill as pickle
from fastprogress.fastprogress import master_bar, progress_bar
import inspect
import torchinfo


In [ ]:
#|hide
import logging

In [ ]:
#|hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
# mpl.rcParams['image.cmap'] = 'gray'

logging.disable(logging.WARNING)

set_seed(42)


In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
_dataset = sample_dataset_dict(_dataset)

In [ ]:
dls = DataLoaders.from_dataset_dict(_dataset, 1024, num_workers=4)

## Core

In [ ]:
#| export
def _lsuv_stats(hook, mod, inp, outp):
    acts = to_cpu(outp)
    hook.mean = acts.mean()
    hook.std = acts.std()

def lsuv_init(model, trainable_module, xb, measurement_module=None,targ_mean=0.5,target_std=1,max_loops=100):
    if hasattr(trainable_module,'bias') or hasattr(trainable_module,'weight'): 
        if measurement_module is None: measurement_module = trainable_module
        hook = Hook(measurement_module, _lsuv_stats)
        while model(xb) is not None and (abs(hook.std-targ_mean)>1e-3) or (abs(hook.mean-targ_mean)>1e-3):
            with torch.no_grad():
                if (abs(hook.std-targ_mean)<1e-3) or (abs(hook.mean-targ_mean)<1e-3): break
                if hasattr(trainable_module,'bias'): trainable_module.bias -= hook.mean - targ_mean
                if hasattr(trainable_module,'weight'): trainable_module.weight.data /= hook.std / target_std
            max_loops-=1
            if max_loops < 1: break
            
        hook.remove()

In [ ]:
model = get_model_conv()
model.to(def_device)
xb = fc.first(dls.train)[0]
for module in model.modules(): lsuv_init(model, module, xb.to(def_device))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()